In [1]:
#Importar librerías necesarias
import json
import numpy as np
import ast
import re
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq


In [2]:
#extraccion del json
rows = []
with open('australian_users_items.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        
df_items = pd.DataFrame(rows)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [3]:
# Desanidado
items_des = []

for index, row in df_items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['items']
    
    for i in items:   
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }
        
        items_des.append(new_row)

df_items_desanidado  = pd.DataFrame(items_des)


In [6]:
#Nulos
df_items_desanidado.isnull().sum()

user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [7]:
df_items_desanidado.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


In [8]:
df_items_desanidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          object
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  int64 
 7   playtime_2weeks   int64 
dtypes: int64(3), object(5)
memory usage: 314.5+ MB


In [9]:
#Busco registros duplicados
duplis = df_items_desanidado[df_items_desanidado.duplicated(keep= "first")]
duplis


,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
164294,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,20,Team Fortress Classic,5,0
164295,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,50,Half-Life: Opposing Force,0,0
164296,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,70,Half-Life,0,0
164297,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,130,Half-Life: Blue Shift,0,0
164298,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,220,Half-Life 2,198,0
...,...,...,...,...,...,...,...,...
4898223,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...,213670,South Park™: The Stick of Truth™,725,0
4898224,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...,221910,The Stanley Parable,53,0
4898225,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...,261030,The Walking Dead: Season Two,253,0
4898226,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...,273110,Counter-Strike Nexon: Zombies,0,0


In [10]:
#Evaluo
df_items_desanidado[(df_items_desanidado["item_name"] == "Half-Life: Blue Shift") & (df_items_desanidado["user_id"] == "Nikiad")]

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
164188,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,130,Half-Life: Blue Shift,0,0
164297,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,130,Half-Life: Blue Shift,0,0


In [11]:
#Son exactamente iguales. Los elimino
df_items_desanidado.drop_duplicates(keep= "first", inplace= True)

In [12]:
#Elimno columnas que no voy a usar
df_items_desanidado.drop(columns=['steam_id',
                           'user_url'], inplace=True)
                         

In [13]:
#Guardado PARQUET
df_items_desanidado.to_parquet('Data/items.parquet', engine='pyarrow')


In [13]:
#Guardado CSV Opcional

#df_items_desanidado.to_csv('Data/items.csv', index=False, encoding="utf-8")